This notebook shows you how to add a folder of finished vasp jobs to the database. This works if you haven't added the structures yet to the database. 

In [1]:
from forge.workflows.vasp_to_db import process_vasp_directory_and_add
from forge.core.database import DatabaseManager


In [2]:
# 1. Your AWS database credentials
db_config = {
    'database': {
        'dbname': 'test_database',  # The name of the database you created on RDS
        'user': 'myless',           # Replace with your RDS username
        'password': 'vcrtiwzr',     # Replace with your RDS password
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432                # Default Postgres port unless you changed it
    }
}

# 2. Instantiate the DatabaseManager
db_manager = DatabaseManager(config_dict=db_config)


In [7]:
process_vasp_directory_and_add(db_manager=db_manager, base_dir='../data/job_gen_8-2025-11-11/')

[INFO] Starting WF-A batch processing of VASP directories in: ../data/job_gen_8-2025-11-11
[INFO] Found 4 potential VASP calculation directories.


Processing VASP dirs (WF-A): 100%|██████████| 4/4 [00:01<00:00,  3.11it/s, Processing: ...2025-11-11/Cr/Cr2_bcc_distorted_idx_2129]


[INFO] Finished WF-A batch processing.
  - Successfully processed (OUTCAR parsed): 4
  - Successfully added to DB (Structure + Calc): 4
  - Skipped as duplicate: 0
  - Failed (Parse Error or Missing Data): 0
  - Failed (DB Add Error): 0


In [8]:
indices = []
# View recent structures
with db_manager.conn.cursor() as cur:
    cur.execute("""
        SELECT structure_id, formula, composition, metadata->>'generation' as gen 
        FROM structures 
        ORDER BY structure_id DESC 
        LIMIT 4
    """)
    print("\nRecent structures:")
    for row in cur.fetchall():
        indices.append(row[0])
        print(f"ID: {row[0]}, Formula: {row[1]}, Composition: {row[2]}, Generation: {row[3]}")


Recent structures:
ID: 22110, Formula: Cr2, Composition: {'Cr': {'at_frac': 1.0, 'num_atoms': 2}}, Generation: 8
ID: 22109, Formula: Cr53, Composition: {'Cr': {'at_frac': 1.0, 'num_atoms': 53}}, Generation: 8
ID: 22108, Formula: Cr252, Composition: {'Cr': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 8
ID: 22107, Formula: Cr2, Composition: {'Cr': {'at_frac': 1.0, 'num_atoms': 2}}, Generation: 8


In [9]:
for i in indices:
    print(f"Structure {i} : {db_manager.get_structure(i)}")
    print(f"Structure Metadata: {db_manager.get_structure_metadata(i)}")
    print(f"{db_manager.get_calculations(i)[0]['energy']}")
    print(f"{db_manager.get_calculations(i)[0]['metadata']}")


Structure 22110 : Atoms(symbols='Cr2', pbc=True, cell=[[2.769994183, -0.051238416, -0.020935818], [0.094092245, 2.72388619, -0.121657161], [0.017253027, -0.06338447, 2.808931297]])
Structure Metadata: {'generation': 8, 'config_type': 'bcc_distorted', 'source_path': '../data/job_gen_8-2025-11-11/Cr/Cr2_bcc_distorted_idx_2129', 'formula_string': 'Cr2', 'composition_str': 'Cr2', 'structure_index': 2129, 'date_added_to_db': '2025-04-13T23:25:25.823475'}
-18.827078
{'nbands': 16, 'parser': 'ase_outcar', 'potcar': 'PAW_PBE Cr_pv 02Aug2007', 'kpoints': [17, 18, 17], 'date_run': '2025.03.30  15:30:12', 'vasp_version': 'vasp.6.4.2 20Jul23', 'resources_used': {'nodes': 1, 'mpi_ranks': 4, 'gpus_detected': True, 'threads_per_rank': 8}, 'calculation_type': 'static'}
Structure 22109 : Atoms(symbols='Cr53', pbc=True, cell=[[8.516023083, 1.19e-07, 5.1e-08], [1.19e-07, 8.516022908, -2.1e-07], [5.1e-08, -2.1e-07, 8.516022849]])
Structure Metadata: {'generation': 8, 'config_type': 'vacancy', 'source_path

In [6]:
for i in indices:
    db_manager.remove_structure(i)

[INFO] Attempting to remove structure ID: 22106 and its calculations...
[INFO] Removed associated calculation IDs: [22101]
[INFO] Successfully removed structure ID: 22106
[INFO] Removal of structure 22106 committed.
[INFO] Attempting to remove structure ID: 22105 and its calculations...
[INFO] Removed associated calculation IDs: [22100]
[INFO] Successfully removed structure ID: 22105
[INFO] Removal of structure 22105 committed.
[INFO] Attempting to remove structure ID: 22104 and its calculations...
[INFO] Removed associated calculation IDs: [22099]
[INFO] Successfully removed structure ID: 22104
[INFO] Removal of structure 22104 committed.
[INFO] Attempting to remove structure ID: 22103 and its calculations...
[INFO] Removed associated calculation IDs: [22098]
[INFO] Successfully removed structure ID: 22103
[INFO] Removal of structure 22103 committed.
